In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from tensorflow.keras.utils import load_img
from tensorflow.keras.preprocessing.image import load_img,ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input
import pandas as pd
import numpy as np
from skimage import exposure, color
import os
from skimage.io import imread

2023-06-07 00:19:54.945197: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-07 00:19:55.115672: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-07 00:19:55.117105: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-07 00:19:56.040017: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# Let's explore the data folder
base_path = './data/'

# Let's define the classes
classes = os.listdir(base_path)

In [4]:
classes

['bottle', 'Empty', 'Glass']

In [5]:
def contrast_stretching(img):
    p2, p98 = np.percentile(img, (3, 90))
    img_rescale = exposure.rescale_intensity(img, in_range=(p2, p98))
    return img_rescale

#Adaptive Histogram Equalization
def AHE(img):
    img_adapteq = exposure.equalize_adapthist(img, clip_limit=0.03)
    return img_adapteq

# Histogram equalization
def HE(img):
    img_eq = exposure.equalize_hist(img)
    return img_eq

def load_image(base_path):
    """it loads all the image into X and the classes in y """
    X_list = []
    y_list = []
    classes = os.listdir(base_path)
    for class_ in classes:
        if not class_.endswith('.DS_Store'):
            files = os.listdir(base_path+class_)
            #print(files)
            for file in files:
                if not file.endswith('.DS_Store'):
                    #print(file)
                    pic = load_img(path=base_path+class_+'/'+f'{file}',target_size=(224,224))
                    numpy_image = np.array(pic)
                    Imagedatagen_input=np.expand_dims(numpy_image.astype("float32")/255,axis=0)
                    processed_image = preprocess_input(numpy_image)
                    X_list.append(processed_image)
                    y_list.append(class_)
                    if class_=='Empty':
                        contrasting_image=ImageDataGenerator(zca_whitening=False,rotation_range=30,
                                                    width_shift_range=0,
                                                    height_shift_range=0.5,
                                                    shear_range=0,
                                                    zoom_range=0,
                                                    fill_mode='nearest',
                                                    horizontal_flip=False,
                                                    vertical_flip=False,
                                                    preprocessing_function=contrast_stretching,
                                                    data_format=K.image_data_format())
                        
                        i=0
                        for img_batch in contrasting_image.flow(Imagedatagen_input,batch_size=6, shuffle=False):
                            print("yes!")
                            for img in img_batch:
                                X_list.append(img)
                                y_list.append(class_)
                                i+=1
                            if i >= 6:
                                break
                    if class_=='bottle':
                        contrasting_image=ImageDataGenerator(zca_whitening=False,rotation_range=30,
                                                    width_shift_range=0,
                                                    height_shift_range=0.5,
                                                    shear_range=0,
                                                    zoom_range=0,
                                                    fill_mode='nearest',
                                                    horizontal_flip=False,
                                                    vertical_flip=False,
                                                    preprocessing_function=contrast_stretching,
                                                    data_format=K.image_data_format())
                        
                        i=0
                        for img_batch in contrasting_image.flow(Imagedatagen_input,batch_size=2, shuffle=False):
                            print("yes!")
                            for img in img_batch:
                                X_list.append(img)
                                y_list.append(class_)
                                i+=1
                            if i >= 2:
                                break  
                    if class_=='Glass':
                        contrasting_image=ImageDataGenerator(zca_whitening=False,rotation_range=30,
                                                    width_shift_range=0,
                                                    height_shift_range=0.5,
                                                    shear_range=0,
                                                    zoom_range=0,
                                                    fill_mode='nearest',
                                                    horizontal_flip=False,
                                                    vertical_flip=False,
                                                    preprocessing_function=contrast_stretching,
                                                    data_format=K.image_data_format())
                        
                        i=0
                        for img_batch in contrasting_image.flow(Imagedatagen_input,batch_size=3, shuffle=False):
                            print("yes!")
                            for img in img_batch:
                                X_list.append(img)
                                y_list.append(class_)
                                i+=1
                            if i >= 3:
                                break                  
                    # AHE_img=ImageDataGenerator(zca_whitening=False,rotation_range=30,
                    #                             width_shift_range=0,
                    #                             height_shift_range=0.5,
                    #                             shear_range=0,
                    #                             zoom_range=0,
                    #                             fill_mode='nearest',
                    #                             horizontal_flip=False,
                    #                             vertical_flip=False,
                    #                             preprocessing_function=AHE,
                    #                             data_format=K.image_data_format())
                    # i=0
                    # for img_batch in AHE_img.flow(Imagedatagen_input,batch_size=9, shuffle=False):
                    #     print("wow!")
                    #     for ahe_img in img_batch:
                    #         X_list.append(ahe_img)
                    #         y_list.append(class_)
                    #         i+=1
                    #     if i >= 9:
                    #         break
                    # HE_img=ImageDataGenerator(zca_whitening=False,rotation_range=30,
                    #                             width_shift_range=0,
                    #                             height_shift_range=0.5,
                    #                             shear_range=0,
                    #                             zoom_range=0,
                    #                             fill_mode='nearest',
                    #                             horizontal_flip=False,
                    #                             vertical_flip=False,
                    #                             preprocessing_function=HE,
                    #                             data_format=K.image_data_format()) 
                    # i=0
                    # for img_batch in HE_img.flow(Imagedatagen_input,batch_size=9, shuffle=False):
                    #     print("yeah!")
                    #     for he_img in img_batch:
                    #         X_list.append(he_img)
                    #         y_list.append(class_)
                    #         i+=1
                    #     if i >= 9:
                    #         break
                                               
                    
                    
        
    # X = np.array(X_list)
    # y = np.array(y_list)
    
    # # shuffle the data
    # shuffler = np.random.permutation(len(X))
    # X = X[shuffler]
    # y = y[shuffler]
    
    return X_list, y_list, classes

In [6]:
X_list, y_list, classes=load_image(base_path)

yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!
yes!


In [7]:
X = np.array(X_list)
y = np.array(y_list)


In [8]:
shuffler = np.random.permutation(len(X))
X = X[shuffler]
y = y[shuffler]

In [9]:
X.shape,len(y),len(classes)

((5403, 224, 224, 3), 5403, 3)

In [10]:
from tensorflow.keras.utils import to_categorical
y_series = pd.Series(y).map({classes[0]:0, classes[1]:1,classes[2]:2})
y = to_categorical(y_series)

In [11]:
y.shape

(5403, 3)

In [12]:
# base_model = keras.applications.vgg16.VGG16(
#     weights='imagenet',input_shape=(224,224,3),
#     include_top=False)

In [43]:
# Early stopping:
early_stop = keras.callbacks.EarlyStopping(
    monitor='val_loss', #the thing we are monitoring
    min_delta=0.005, #the minimum change in the quantity that we want for the model to train for another epoch
    patience=2, #number of epochs with no improvement needed for the model to stop
    verbose=1, #0 is silent, 1 means a message is displayed when something happens
    mode='auto'
)



In [44]:
# K.clear_session()
# new_model = keras.models.Sequential([
    
#     # Base model
#     base_model,
    
#     # Flatten layer
#     keras.layers.Flatten(),
    
#     #keras.layers.Dropout(.5),
#     # fc1 fully connected layer
#     #keras.layers.Dense(units=100, activation=keras.activations.relu, name='fc1'),
    
#     # fc2 fully connected layer
#     #keras.layers.Dense(units=100, activation=keras.activations.relu, name='fc2'),
    
#     # output layer
#     keras.layers.Dense(units=3, activation=keras.activations.relu, name='output')

# ])

In [45]:
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Activation, Flatten, Dropout, BatchNormalization

In [46]:
# Clear session and build above model:
K.clear_session()
new_model = keras.models.Sequential([
    Conv2D(filters = 8, kernel_size = (6,6),padding = 'valid',activation = 'relu' ,input_shape = (224,224,3)),
    #Conv2D(filters = 16, kernel_size = (5,5), padding = 'valid', activation = 'relu'),
    MaxPooling2D(pool_size = (5,5),padding = 'valid'),

    
    # Conv2D(filters = 16, kernel_size = (3,3), strides = (1,1), padding = 'valid', activation = 'relu'),
    # Conv2D(filters = 16, kernel_size = (3,3), strides = (1,1), padding = 'valid', activation = 'relu'),
    # MaxPooling2D(pool_size = (2,2),strides=(2,2),padding = 'valid'),
    
    # Conv2D(filters = 16, kernel_size = (3,3), strides = (1,1), padding = 'valid', activation = 'relu'),
    # Conv2D(filters = 16, kernel_size = (3,3), strides = (1,1), padding = 'valid', activation = 'relu'),
    # MaxPooling2D(pool_size = (2,2),strides=(2,2),padding = 'valid'),

    # Conv2D(filters = 16, kernel_size = (3,3), strides = (1,1), padding = 'valid', activation = 'relu'),
    # Conv2D(filters = 16, kernel_size = (3,3), strides = (1,1), padding = 'valid', activation = 'relu'),
    # MaxPooling2D(pool_size = (2,2),strides=(2,2),padding = 'valid'),    
    Flatten(),
    #Fully connected layers:
        
    Dropout(0.005),
    # fc1 fully connected layer
    Dense(units=50, activation='relu', name='fc1'),
    
    # fc2 fully connected layer
    #Dense(units=30, activation='relu', name='fc2'),
    # _Dropout(0.2), - optional
    
    Dense(units=3, activation=keras.activations.softmax, name='output')   
])

In [47]:
new_model.summary()
new_model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adam(learning_rate=0.008),
                 metrics=['categorical_accuracy'])

history = new_model.fit(X,y,batch_size=300, epochs=30, validation_split=.2,callbacks=[early_stop])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 219, 219, 8)       872       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 43, 43, 8)        0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 14792)             0         
                                                                 
 dropout (Dropout)           (None, 14792)             0         
                                                                 
 fc1 (Dense)                 (None, 50)                739650    
                                                                 
 output (Dense)              (None, 3)                 153       
                                                        